In [26]:
import re

text = r"\boxed{\text{tr}(AB) = \text{tr}(BA)}"
match = re.search(r"\\boxed{(.*)}", text)
if match:
    print(match.group(1))  # 输出: \text{tr}(AB) = \text{tr}(BA)

\text{tr}(AB) = \text{tr}(BA)


In [32]:
#区分一下reason数据集中有多少可以验证的数据
import json
import regex as re
import os

file_path = "/home/mth/project_llm/mini_llm/data/origin_data/distill_r1_110k.jsonl"
output_file_path_1 = "/home/mth/project_llm/mini_llm/data/origin_data/distill_r1_validate_sft.jsonl"
output_file_path_2 = "/home/mth/project_llm/mini_llm/data/origin_data/distill_r1_general_sft.jsonl"

def ensure_file_exists(file_path):
    if not os.path.exists(file_path):
        with open(file_path, 'w', encoding='utf-8') as f:
            pass  # 创建空文件

pattern = r'\\boxed\{.*?\}'
validate_pattern = r"\\boxed{(.*)}"

output_datas_1 = []
output_datas_2 = []


with open(file_path, mode="r", encoding="utf-8") as file:
    for line in file:
        data = json.loads(line.strip()) 
        if re.search(pattern, data['content']):
            match = re.search(r"\\boxed{(.*)}", data['content'])
            validate = match.group(1)
            if data['score'] == 10:
                new_data = {
                    "conversations": [
                        {"role": "user", "content": f"{data['input']}"},
                        {"role": "assistant", "content": f"<think>\n{data['reasoning_content']}\n</think>\n{data['content']}"}
                    ],
                    "validate":f"{validate}"
                }
                output_datas_1.append(new_data)
        else:
            if data['score'] >=6:
                new_data = {
                    "conversations": [
                        {"role": "user", "content": f"{data['input']}"},
                        {"role": "assistant", "content": f"<think>\n{data['reasoning_content']}\n</think>\n{data['content']}"}
                    ],
                    "validate":""
                }
                output_datas_2.append(new_data)

print(f"可以校验的数据共计：{len(output_datas_1)}行。不可校验的数据共计：{len(output_datas_2)}行")
with open(output_file_path_1, mode="w", encoding="utf-8") as file:
    json.dump(output_datas_1, file, ensure_ascii=False, indent=4)

with open(output_file_path_2, mode="w", encoding="utf-8") as file:
    json.dump(output_datas_2, file, ensure_ascii=False, indent=4)




可以校验的数据共计：36233行。不可校验的数据共计：69949行


In [ ]:
# 处理医疗问答数据集

import json
import regex as re
import os

file_path = "/home/mth/project_llm/mini_llm/data/origin_data/ChatMed_TCM-v0.2.json"
output_file_path = "/home/mth/project_llm/mini_llm/data/origin_data/ChatMed_TCM_sft.jsonl"

output_datas = []


with open(file_path, mode="r", encoding="utf-8") as file:
    for line in file:
        data = json.loads(line.strip()) 
        query = data['query'].split("要求：")[0]
    
            
        new_data = {
                    "conversations": [
                        {"role": "user", "content": f"{query}"},
                        {"role": "assistant", "content": f"{data['response']}"}
                    ]
                }
        output_datas.append(new_data)

print(f"共计医学问答数据：{len(output_datas)}行")
with open(output_file_path, mode="w", encoding="utf-8") as file:
    json.dump(output_datas, file, ensure_ascii=False, indent=4)


共计医学问答数据：112565行


In [16]:
import pandas as pd

# 文件路径
file_name_1 = "/home/mth/project_llm/mini_llm/data/origin_data/中医辩证数据集/test_set.csv"
file_name_2 = "/home/mth/project_llm/mini_llm/data/origin_data/中医辩证数据集/TCM_syndrome_differentiation.csv"
output_path = "/home/mth/project_llm/mini_llm/data/origin_data/中医辩证数据集/TCM_syndrome_differentiation_new.csv"
output_count_path = "/home/mth/project_llm/mini_llm/data/origin_data/中医辩证数据集/syndrome_count_filtered_new.csv"

# 读取两个文件
df1 = pd.read_csv(file_name_2)
# df2 = pd.read_csv(file_name_2)

# 合并数据
# merged_df = pd.concat([df1, df2], ignore_index=True)
merged_df = df1
# 统计每个syndrome的病例数量
syndrome_count = merged_df.groupby('syndrome').size().reset_index(name='count')

# 过滤掉病例数不足30个的syndrome
filtered_syndrome_count = syndrome_count[syndrome_count['count'] >= 100]

# 打印过滤后的syndrome数量
print(f"过滤后的syndrome数量:\n{filtered_syndrome_count}")

# 根据过滤后的syndrome列表，保留满足条件的数据
filtered_merged_df = merged_df[merged_df['syndrome'].isin(filtered_syndrome_count['syndrome'])]

# 保存过滤后的合并数据
filtered_merged_df.to_csv(output_path, index=False)
print(f"过滤后的合并数据已保存到: {output_path}")

# 保存过滤后的syndrome病例数量统计
filtered_syndrome_count.to_csv(output_count_path, index=False)
print(f"过滤后的syndrome病例数量统计已保存到: {output_count_path}")


过滤后的syndrome数量:
   syndrome  count
1     寒凝血瘀证    102
6     心血不足证    109
7     心血瘀阻证    441
9     正虚瘀结证   1907
13    气滞血瘀证   4720
16    气虚血瘀证   7783
19    气血亏虚证    262
21    气阴两虚证    824
23    湿热下注证   4394
24    湿热内蕴证    116
25    湿热瘀阻证    627
26    湿热蕴结证    507
27    湿热阻络证    120
28    热毒壅结证    868
31    痰浊瘀阻证    304
32    痰浊蒙窍证    233
33    痰湿中阻证    188
34  痰湿蒙闭心窍证    175
35    痰湿蕴肺证   1797
38    痰瘀互结证    293
39    痰瘀痹阻证    145
42    瘀血阻络证    270
45    肝经湿热证    170
46    肝肾不足证    515
47    肝肾亏虚证    907
48    肝肾阴虚证    196
50    肝胃不和证    540
51    肝胃郁热证   1897
56    肝阳上亢证    206
62      肾虚证    584
63    脾肾两虚证    894
65    脾胃不和证    146
68     脾胃虚证    169
69    脾胃阳虚证    540
73      血热证    162
74    血瘀痰滞证    180
76    阳虚水泛证    495
80    阴虚火旺证    177
82    阴虚血瘀证    130
83    阴虚阳亢证    305
84    风寒外袭证    400
85   风寒湿痹阻证    122
87    风寒袭肺证    507
92    风痰上攻证    402
93    风痰入络证    318
94    风痰阻络证   3148
过滤后的合并数据已保存到: /home/mth/project_llm/mini_llm/data/origin_data/中医辩证数据集/TCM_syndrome_differen

In [39]:
from sklearn.model_selection import train_test_split

output_path = "/home/mth/project_llm/mini_llm/data/origin_data/中医辩证数据集/TCM_syndrome_differentiation.csv"
train_output_path = "/home/mth/project_llm/mini_llm/data/origin_data/中医辩证数据集/TCM_syndrome_differentiation_train_set.csv"
test_output_path = "/home/mth/project_llm/mini_llm/data/origin_data/中医辩证数据集/TCM_syndrome_differentiation_test_set.csv"

filtered_merged_df = pd.read_csv(output_path)

# 将每个syndrome随机抽取6个作为训练集，剩下的作为测试集
train_list = []
test_list = []

for syndrome in filtered_merged_df['syndrome'].unique():
    # 提取当前syndrome的数据
    syndrome_data = filtered_merged_df[filtered_merged_df['syndrome'] == syndrome]
    
    # 随机抽取6个作为训练集，剩余的作为测试集
    train_data, test_data = train_test_split(syndrome_data, train_size=6, random_state=42)
    
    # 添加到训练集和测试集列表
    train_list.append(train_data)
    test_list.append(test_data)

# 合并训练集和测试集
train_set = pd.concat(train_list, ignore_index=True)
test_set = pd.concat(test_list, ignore_index=True)

# 保存训练集和测试集
train_set.to_csv(train_output_path, index=False)
test_set.to_csv(test_output_path, index=False)
print(f"训练集已保存到: {train_output_path}")
print(f"测试集已保存到: {test_output_path}")

训练集已保存到: /home/mth/project_llm/mini_llm/data/origin_data/中医辩证数据集/TCM_syndrome_differentiation_train_set.csv
测试集已保存到: /home/mth/project_llm/mini_llm/data/origin_data/中医辩证数据集/TCM_syndrome_differentiation_test_set.csv


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

# 文件路径
output_path = "/home/mth/project_llm/mini_llm/data/origin_data/中医辩证数据集/TCM_syndrome_differentiation.csv"
train_output_path = "/home/mth/project_llm/mini_llm/data/origin_data/中医辩证数据集/TCM_syndrome_differentiation_train.csv"
val_output_path = "/home/mth/project_llm/mini_llm/data/origin_data/中医辩证数据集/TCM_syndrome_differentiation_val.csv"
test_output_path = "/home/mth/project_llm/mini_llm/data/origin_data/中医辩证数据集/TCM_syndrome_differentiation_test.csv"

# 读取数据
filtered_merged_df = pd.read_csv(output_path)

# 抽取比例
train_ratio = 0.7
val_ratio = 0.1
test_ratio = 0.3

# 存放每个类别的训练/验证/测试数据
train_list = []
val_list = []
test_list = []

# 每个syndrome单独划分
for syndrome in filtered_merged_df['syndrome'].unique():
    syndrome_data = filtered_merged_df[filtered_merged_df['syndrome'] == syndrome]
    
    # 先按训练+剩余
    train_data, temp_data = train_test_split(syndrome_data, train_size=train_ratio, random_state=42)

    # 再把剩余的分成验证集和测试集
    val_data, test_data = train_test_split(temp_data, train_size=val_ratio / (val_ratio + test_ratio), random_state=42)

    # 加到列表
    train_list.append(train_data)
    val_list.append(val_data)
    test_list.append(test_data)

# 合并
train_set = pd.concat(train_list, ignore_index=True)
val_set = pd.concat(val_list, ignore_index=True)
test_set = pd.concat(test_list, ignore_index=True)

# 保存
train_set.to_csv(train_output_path, index=False)
val_set.to_csv(val_output_path, index=False)
test_set.to_csv(test_output_path, index=False)

print(f"训练集已保存到: {train_output_path}")
print(f"验证集已保存到: {val_output_path}")
print(f"测试集已保存到: {test_output_path}")


训练集已保存到: /home/mth/project_llm/mini_llm/data/origin_data/中医辩证数据集/TCM_syndrome_differentiation_train.csv
验证集已保存到: /home/mth/project_llm/mini_llm/data/origin_data/中医辩证数据集/TCM_syndrome_differentiation_val.csv
测试集已保存到: /home/mth/project_llm/mini_llm/data/origin_data/中医辩证数据集/TCM_syndrome_differentiation_test.csv


In [2]:
#处理中医书籍数据集

import json
import csv
import ast

file_path = "/home/mth/project_llm/mini_llm/data/origin_data/PT_University_TCM_books.csv"
output_file_path = "/home/mth/project_llm/mini_llm/data/origin_data/TCM_book_sft.jsonl"

output_datas = []


with open(file_path, mode="r", encoding="utf-8") as file:
    csvreader = csv.reader(file)
    
        # 如果有标题行，可以跳过标题行
    next(csvreader)  # 跳过标题行

        # 遍历每一行
    for line in csvreader:
        new_data = {
                    "conversations": [
                        {"role": "user", "content": f""},
                        {"role": "assistant", "content": f"{line[0]}"}
                    ]
                }
        output_datas.append(new_data)

print(f"共计数据：{len(output_datas)}行")
with open(output_file_path, mode="w", encoding="utf-8") as file:
    json.dump(output_datas, file, ensure_ascii=False, indent=4)


共计数据：23753行


In [7]:
#处理中医问答数据集

import json
import csv
import ast

file_path = "/home/mth/project_llm/mini_llm/data/origin_data/PT_dialogue.csv"
output_file_path = "/home/mth/project_llm/mini_llm/data/origin_data/TCM_dialogue_sft.jsonl"

output_datas = []


with open(file_path, mode="r", encoding="utf-8") as file:
    csvreader = csv.reader(file)
    
        # 如果有标题行，可以跳过标题行
    next(csvreader)  # 跳过标题行

        # 遍历每一行
    for line in csvreader:
        line = ast.literal_eval(line[0])
        pass
        new_data = {
                    "conversations": [
                        {"role": "user", "content": f"{line[2]}"},
                        {"role": "assistant", "content": f"{line[3]}"}
                    ]
                }
        output_datas.append(new_data)

print(f"共计医学问答数据：{len(output_datas)}行")
with open(output_file_path, mode="w", encoding="utf-8") as file:
    json.dump(output_datas, file, ensure_ascii=False, indent=4)

共计医学问答数据：792099行


In [3]:
#处理lora数据集
import json
import regex as re
import random

file_path = "/home/mth/project_llm/mini_llm/data/origin_data/Medical_consultation_data_SFT/med_6part/med_6.json"
file_path_1 = "/home/mth/project_llm/mini_llm/data/origin_data/Medical_consultation_data_SFT/med_zh/train_zh.json"
output_file_path = "/home/mth/project_llm/mini_llm/data/origin_data/ChatMed_lora.jsonl"

output_datas = []


with open(file_path, mode="r", encoding="utf-8") as file:
    for line in file:
        data = json.loads(line.strip()) 
    
            
        new_data = {
                    "conversations": [
                        {"role": "user", "content": f"{data['instruction']}"},
                        {"role": "assistant", "content": f"{data['output']}"}
                    ]
                }
        output_datas.append(new_data)

with open(file_path, mode="r", encoding="utf-8") as file:
    for line in file:
        data = json.loads(line.strip()) 
    
            
        new_data = {
                    "conversations": [
                        {"role": "user", "content": f"{data['instruction']}"},
                        {"role": "assistant", "content": f"{data['output']}"}
                    ]
                }
        output_datas.append(new_data)

print(f"共计医学问答数据：{len(output_datas)}行")
with open(output_file_path, mode="w", encoding="utf-8") as file:
    random.shuffle(output_datas)
    for data in output_datas:
        file.write(json.dumps(data, ensure_ascii=False) + "\n")

共计医学问答数据：280280行


In [5]:
# 处理工匠数据集

import json
import regex as re
import os

file_path = "/home/mth/project_llm/mini_llm/data/origin_data/sft_data_zh.jsonl"
output_file_path = "/home/mth/project_llm/mini_llm/data/origin_data/gongjiang_data_sft.jsonl"

output_datas = []


with open(file_path, mode="r", encoding="utf-8") as file:
    for line in file:
        data = json.loads(line.strip()) 
        conversations = [
            {"role": "user", "content": f"{data['input']}"},
            {"role": "assistant", "content": f"{data['output']}"}
            ]

        if data['history']:
            for conversation in data['history']:
                conversations.append({"role": "user", "content": f"{conversation[0]}"})
                conversations.append({"role": "assistant", "content": f"{conversation[1]}"})
        pass
        new_data = {
                    "conversations": conversations
                }
        output_datas.append(new_data)

print(f"共计问答数据：{len(output_datas)}行")
with open(output_file_path, mode="w", encoding="utf-8") as file:
    json.dump(output_datas, file, ensure_ascii=False, indent=4)

共计问答数据：11381621行


KeyboardInterrupt: 

In [1]:
import json

# 文件路径
file_path = "/home/mth/project_llm/mini_llm/data/pretrain_data/pretrain.jsonl"

# 统计文件总行数
def count_lines(file_path):
    with open(file_path, mode="r", encoding="utf-8") as file:
        return sum(1 for _ in file)

# 使用生成器读取前 n 行数据
def load_first_n_lines(file_path, n=10):
    with open(file_path, mode="r", encoding="utf-8") as file:
        for i, line in enumerate(file):
            if i >= n:  # 只读取前n行
                break
            line = line.strip()  # 去除行首尾的空白字符
            if not line:  # 跳过空行
                continue
            try:
                yield json.loads(line)  # 解析每一行的JSON数据
            except json.JSONDecodeError:
                print(f"跳过无效行: {line}")

# 获取总行数
total_lines = count_lines(file_path)

# 读取前10行数据
data = list(load_first_n_lines(file_path, n=10))

print(f"成功加载 {len(data)} 条数据, 共计 {total_lines} 条数据")

for d in data:
    print(d)


成功加载 10 条数据, 共计 4109472 条数据
{'tokens': [1, 62, 1313, 7177, 1395, 1091, 6840, 1593, 266, 2377, 3088, 5541, 882, 2251, 881, 547, 3138, 3607, 5176, 268, 331, 1193, 882, 2251, 1538, 1091, 881, 547, 882, 2251, 2515, 523, 2466, 1827, 266, 1419, 390, 1197, 268, 304, 1091, 881, 547, 882, 2251, 2237, 264, 3138, 5176, 727, 7173, 3088, 474, 4, 3921, 264, 2734, 2579, 3614, 2319, 268, 2344, 291, 644, 62, 1313, 7177, 1395, 1091, 6840, 1593, 592, 266, 2377, 3088, 5541, 882, 2251, 881, 547, 3607, 5176, 264, 1242, 331, 1193, 882, 2251, 266, 1538, 6183, 389, 881, 547, 510, 523, 2466, 1827, 266, 1419, 390, 1197, 268, 1532, 693, 9070, 881, 547, 331, 1193, 882, 2251, 2237, 264, 5176, 727, 7173, 2329, 268, 1280, 264, 308, 515, 405, 4787, 2344, 755, 3437, 5467, 4926, 268, 2344, 727, 3528, 1524, 881, 547, 882, 2251, 503, 2515, 2466, 1827, 264, 4896, 880, 3812, 2466, 1827, 4684, 2377, 3088, 264, 368, 2415, 474, 1242, 331, 2377, 3088, 291, 4656, 523, 2466, 1827, 266, 1419, 390, 1197, 268, 4896, 644, 264, 881, 5

In [21]:
import json


file_path = "/home/mth/project_llm/mini_llm/data/origin_data/sft.jsonl"
count = 0
with open(file_path, mode="r", encoding="utf-8") as file:
      for line in file:
            data = json.loads(line.strip())
            count += int(data['tokens'])


print(f"共计:{count}tokens")
            

共计:3060824071tokens


In [1]:
import json

file_path = "/home/mth/project_llm/mini_llm/data/origin_data/deduped_sft.jsonl"
file_path_1 = "/home/mth/project_llm/mini_llm/data/origin_data/sft_512.jsonl"
file_path_2 = "/home/mth/project_llm/mini_llm/data/origin_data/sft_4k.jsonl"
file_path_3 = "/home/mth/project_llm/mini_llm/data/origin_data/sft_rl.jsonl"

buffer = []
count_1 = 0  # 计数存入file_path_1的数量
count_2 = 0  # 计数存入file_path_2的数量
count_3 = 0  # 计数存入file_path_3的数量

k_data = []

with open(file_path, mode="r", encoding="utf-8") as file:
    for line in file:
        data = json.loads(line.strip())

        if data['conversations'][0]['content']:
            buffer.append(data)
            if len(buffer) > 10000:
                for d in buffer:
                    if 'validate' in d and d['validate']:
                        # with open(file_path_3, 'a', encoding='utf-8') as f:
                        #     f.write(json.dumps(d, ensure_ascii=False) + "\n")
                        count_3 += 1
                    elif d['tokens'] < 490 and "<think>" not in d['conversations'][1]['content']:
                        # with open(file_path_1, 'a', encoding='utf-8') as f:
                        #     f.write(json.dumps(d, ensure_ascii=False) + "\n")
                        count_1 += 1
                    elif d['tokens'] > 1000 and "<think>" not in d['conversations'][1]['content']:
                        k_data.append(d)
                        count_2 += 1
                buffer = []

# 处理剩余的buffer
if buffer:
    for d in buffer:
        if 'validate' in d and d['validate']:
            # with open(file_path_3, 'a', encoding='utf-8') as f:
            #     f.write(json.dumps(d, ensure_ascii=False) + "\n")
            count_3 += 1
        elif d['tokens'] < 490 and "<think>" not in d['conversations'][1]['content']:
            # with open(file_path_1, 'a', encoding='utf-8') as f:
            #     f.write(json.dumps(d, ensure_ascii=False) + "\n")
            count_1 += 1
        elif d['tokens'] > 1000 and "<think>" not in d['conversations'][1]['content']:
            # with open(file_path_2, 'a', encoding='utf-8') as f:
            #     f.write(json.dumps(d, ensure_ascii=False) + "\n")
            k_data.append(d)
            count_2 += 1


with open(file_path_2, 'a', encoding='utf-8') as f:
    k_data.sort(key=lambda x:x['tokens'])
    for data in k_data:
        f.write(json.dumps(data, ensure_ascii=False) + "\n")
# 打印各文件写入的数量
print(f"File 1 (tokens < 490): {count_1}")
print(f"File 2 (tokens > 2000): {count_2}")
print(f"File 3 (validate exists): {count_3}")


File 1 (tokens < 490): 8364950
File 2 (tokens > 2000): 497277
File 3 (validate exists): 6797


In [1]:
import json

def split_jsonl_file(input_file, num_parts=13):
    # 读取原始 JSONL 文件
    with open(input_file, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    # 计算每个子文件包含的行数
    total_lines = len(lines)
    lines_per_file = total_lines // num_parts
    remainder = total_lines % num_parts

    # 分割文件并写入子文件
    start_idx = 0
    for i in range(num_parts):
        end_idx = start_idx + lines_per_file + (1 if i < remainder else 0)  # 分配剩余的行
        output_file = f"/home/mth/project_llm/mini_llm/data/pretrain_data/pretrain_{i}.jsonl"  # 子文件名按数字命名
        with open(output_file, 'w', encoding='utf-8') as f_out:
            f_out.writelines(lines[start_idx:end_idx])
        start_idx = end_idx
        print(f"Created: {output_file} with {end_idx - start_idx} lines.")

# 使用示例
input_file = '/home/mth/project_llm/mini_llm/data/pretrain_data/pretrain.jsonl'  # 替换为你的原始文件路径
split_jsonl_file(input_file, num_parts=10)


Created: /home/mth/project_llm/mini_llm/data/pretrain_data/pretrain_0.jsonl with 0 lines.
Created: /home/mth/project_llm/mini_llm/data/pretrain_data/pretrain_1.jsonl with 0 lines.
Created: /home/mth/project_llm/mini_llm/data/pretrain_data/pretrain_2.jsonl with 0 lines.
Created: /home/mth/project_llm/mini_llm/data/pretrain_data/pretrain_3.jsonl with 0 lines.
Created: /home/mth/project_llm/mini_llm/data/pretrain_data/pretrain_4.jsonl with 0 lines.
Created: /home/mth/project_llm/mini_llm/data/pretrain_data/pretrain_5.jsonl with 0 lines.
Created: /home/mth/project_llm/mini_llm/data/pretrain_data/pretrain_6.jsonl with 0 lines.
Created: /home/mth/project_llm/mini_llm/data/pretrain_data/pretrain_7.jsonl with 0 lines.
Created: /home/mth/project_llm/mini_llm/data/pretrain_data/pretrain_8.jsonl with 0 lines.
Created: /home/mth/project_llm/mini_llm/data/pretrain_data/pretrain_9.jsonl with 0 lines.


In [4]:
import json
import random

file_path = "/home/mth/project_llm/mini_llm/data/origin_data/中医辩证数据集/TCM_syndrome_differentiation_train_reason.jsonl"
file_path_1 = "/home/mth/project_llm/mini_llm/data/origin_data/TCM_sft_rl.jsonl"

buffer = []
k_data = []

with open(file_path, mode="r", encoding="utf-8") as file:
    for line in file:
        data = json.loads(line.strip())

        if data['conversations'][0]['content']:
            buffer.append(data)
            if len(buffer) > 1000:
                for d in buffer:
                    systme_prompt = {"role": "system", "content": "user和assistant之间的对话。user提出问题，assistant解决问题。assistant首先在头脑中思考推理过程，然后向用户提供答案。推理过程分别包含在<think> </think>标签中，答案包含在<answer> </answer>标签中，即<think>\n reasoning process here\n</think>\n<answer>\n answer here \n</answer>。"}

                    data_1 = d['conversations'][0]['content']
                    data_1 = {"role": "user", "content": f"您必须将您的答案放在<answer> </answer>标签中，即<answer>该问题的答案</answer>。你的最终答案将通过\\boxed{{}}标签自动提取出来。\n以下是具体的问题：\n{data_1}"}
                    
                    data_2 = d['conversations'][1]

                    data_2_1 = data_2['content'].split("</think>")[0].strip()
                    data_2_2 = data_2['content'].split("</think>")[1].strip()

                    data_2 = {"role": "assistant", "content": f"{data_2_1}\n</think>\n<answer>\n{data_2_2}\n</answer>"}

                    d['conversations'] = [systme_prompt, data_1, data_2]

               
                    k_data.append(d)
                       
                buffer = []

# 处理剩余的buffer
if buffer:
    for d in buffer:
        systme_prompt = {"role": "system", "content": "user和assistant之间的对话。user提出问题，assistant解决问题。assistant首先在头脑中思考推理过程，然后向用户提供答案。推理过程分别包含在<think> </think>标签中，答案包含在<answer> </answer>标签中，即<think>\n reasoning process here\n</think>\n<answer>\n answer here \n</answer>。"}

        data_1 = d['conversations'][0]['content']
        data_1 = {"role": "user", "content": f"您必须将您的答案放在<answer> </answer>标签中，即<answer>该问题的答案</answer>。你的最终答案将通过\\boxed{{}}标签自动提取出来。\n以下是具体的问题：\n{data_1}"}
                    
        data_2 = d['conversations'][1]

        data_2_1 = data_2['content'].split("</think>")[0].strip()
        data_2_2 = data_2['content'].split("</think>")[1].strip()

        data_2 = {"role": "assistant", "content": f"{data_2_1}\n</think>\n<answer>\n{data_2_2}\n</answer>"}

        d['conversations'] = [systme_prompt, data_1, data_2]

               
        k_data.append(d)       



with open(file_path_1, 'a', encoding='utf-8') as f:
    random.shuffle(k_data)
    for data in k_data:
        f.write(json.dumps(data, ensure_ascii=False) + "\n")


In [5]:
import json
from sklearn.model_selection import train_test_split
import csv
import random

output_path_1 = "/home/mth/project_llm/mini_llm/data/origin_data/grpo_tcm_train.jsonl"
output_path_2 = "/home/mth/project_llm/mini_llm/data/origin_data/grpo_tcm_test.jsonl"
file_path_1 = "/home/mth/project_llm/mini_llm/data/origin_data/中文数学题目/math23k_test.json"
file_path_2 = "/home/mth/project_llm/mini_llm/data/origin_data/中文数学题目/math23k_train.json"
file_path_3 = "/home/mth/project_llm/mini_llm/data/origin_data/中医辩证数据集/TCM_syndrome_differentiation_train.csv"

buffer = []



def write_jsonl_file(data, file_path):
    with open(file_path, 'w', encoding='utf-8') as f:
        for entry in data:
            f.write(json.dumps(entry, ensure_ascii=False) + '\n')

systme_prompt = {"role": "system", "content": "user和assistant之间的对话。user提出问题，assistant解决问题。assistant首先在头脑中思考推理过程，然后向用户提供答案。推理过程分别包含在<think> </think>标签中，答案包含在<answer> </answer>标签中，即<think>\n reasoning process here\n</think>\n<answer>\n answer here \n</answer>。"}


data = []
# with open(file_path_1, 'r', encoding='utf-8') as f:
#         datas = json.load(f)
#         for d in datas:
#             data_1 = {"role": "user", "content": f"您必须将您的答案放在<answer> </answer>标签中，即<answer>该问题的答案</answer>。你的最终答案将通过\\boxed{{}}标签自动提取出来。\n以下是具体的问题：\n{d['original_text']}"}
#             answer = d['ans']

#             data.append({
#                 'prompt': [systme_prompt, data_1],
#                 'answer':answer
#             })

# with open(file_path_2, 'r', encoding='utf-8') as f:
#         datas = json.load(f)
#         for d in datas:
#             data_1 = {"role": "user", "content": f"您必须将您的答案放在<answer> </answer>标签中，即<answer>该问题的答案</answer>。你的最终答案将通过\\boxed{{}}标签自动提取出来。\n以下是具体的问题：\n{d['original_text']}"}
#             answer = d['ans']

#             data.append({
#                 'prompt': [systme_prompt, data_1],
#                 'answer':answer
#             })

with open(file_path_3, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            data_1 = {"role": "user", "content": f"您必须将您的答案放在<answer> </answer>标签中，即<answer>该问题的答案</answer>。你的最终答案将通过\\boxed{{}}标签自动提取出来。\n以下是具体的问题：\n你是一名专业的中医医生，可以根据病人的病例信息给出病人的辩证。病人的病例：{row['description']}"}
            answer = row['syndrome']

            data.append({
                'prompt': [systme_prompt, data_1],
                'answer':answer
            })


# 打乱数据
random.shuffle(data)

    # 划分训练集和测试集，比例为9:1
# train_data, test_data = train_test_split(data, test_size=0.1, random_state=42)

write_jsonl_file(data, output_path_1)
# write_jsonl_file(test_data, output_path_2)
             
    





In [2]:
import pandas as pd

# 读取原始CSV文件
df = pd.read_csv('/home/mth/project_llm/mini_llm/data/origin_data/中医辩证数据集/TCM_syndrome_differentiation_new.csv')

# 每个syndrome最多保留300条记录
df_limited = df.groupby('syndrome').head(300)

# 保存到新的CSV文件
df_limited.to_csv('TCM_syndrome_differentiation.csv', index=False)

print(f"筛选完成，共保存{len(df_limited)}条记录到output.csv")


筛选完成，共保存11171条记录到output.csv
